# Basic helping functions

In [1]:
from crypto import binary, hexadecimal, bytes_to_bin, bytes_to_hex

In [2]:
message = b"simple message"

bin_repr = bytes_to_bin(message, pre="")
hex_repr = bytes_to_hex(message, pre="")

print(f"message:\n{message}\nlen bytes: {len(message)}\n")
print(f"message in binary:\n{bin_repr}\nlen bits: {len(bin_repr)}\n")
print(f"message in hexadecimal:\n{hex_repr}\nlen hex:{len(hex_repr)}\n")

message:
b'simple message'
len bytes: 14

message in binary:
0111001101101001011011010111000001101100011001010010000001101101011001010111001101110011011000010110011101100101
len bits: 112

message in hexadecimal:
73696d706c65206d657373616765
len hex:28



# Stream and Block ciphers

According to the lenght of the ciphertext compared do the lenght of the original message we can classify ciphers as being stream or block.

# Stream ciphers


<img src="img/stream_cipher.png" style="width:900px"/>

The ciphertext has the same lenght as the plaintext, if we add one more bit to the plaintext that would result in one more bit in the plaintext. Let's use a stream cipher called [ChaCha20](https://tools.ietf.org/html/draft-strombergson-chacha-test-vectors-00), you can find a python implementation [here](https://asecuritysite.com/encryption/chacha).

In [3]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

# we need a key and a nonce generated using urandom from your os
secret_key = os.urandom(32)
nonce = os.urandom(16)
algorithm = algorithms.ChaCha20(secret_key, nonce)

# we define here the cipher
chachacipher = Cipher(algorithm, mode=None, backend=default_backend())
encryptor = chachacipher.encryptor()
decryptor = chachacipher.decryptor()

In [4]:
message = b"A super secret message"
ctx = encryptor.update(message)
ptx = decryptor.update(ctx)

In [5]:
for message_len in range(32):
    message = str.encode("a"*message_len)
    ctx = encryptor.update(message)
    print(f"message_len: {message_len}, ciphertext_len: {len(ctx)}")

message_len: 0, ciphertext_len: 0
message_len: 1, ciphertext_len: 1
message_len: 2, ciphertext_len: 2
message_len: 3, ciphertext_len: 3
message_len: 4, ciphertext_len: 4
message_len: 5, ciphertext_len: 5
message_len: 6, ciphertext_len: 6
message_len: 7, ciphertext_len: 7
message_len: 8, ciphertext_len: 8
message_len: 9, ciphertext_len: 9
message_len: 10, ciphertext_len: 10
message_len: 11, ciphertext_len: 11
message_len: 12, ciphertext_len: 12
message_len: 13, ciphertext_len: 13
message_len: 14, ciphertext_len: 14
message_len: 15, ciphertext_len: 15
message_len: 16, ciphertext_len: 16
message_len: 17, ciphertext_len: 17
message_len: 18, ciphertext_len: 18
message_len: 19, ciphertext_len: 19
message_len: 20, ciphertext_len: 20
message_len: 21, ciphertext_len: 21
message_len: 22, ciphertext_len: 22
message_len: 23, ciphertext_len: 23
message_len: 24, ciphertext_len: 24
message_len: 25, ciphertext_len: 25
message_len: 26, ciphertext_len: 26
message_len: 27, ciphertext_len: 27
message_len:

The lenght of the ciphertext is the same as the lenght of the plaintext in stream ciphers.

# Bock ciphers

Data is encrypted in blocks of certain amount of bytes, for instance 16 bytes. In general a block cipher that encodes with size 16 calculates ciphertexts of size multiple 16.

<img src="img/block_cipher.png" style="width:1100px"/>

## Padding a message

Most of the times the lenght of the message is not a multiple of the block size so we need to "pad" the message to have the required length. A common padding function is [PKCS7](https://en.wikipedia.org/wiki/Padding_(cryptography)). Basically what PKCS7 does is appendinng a list of bytes with the same value corresponding to the number of bytes needed to complete the block.


We will use PKCS7 it in the next example:

In [6]:
message = b"Cryptography is a complex subject after all..."
bin_repr = bytes_to_bin(message, pre="")
hex_repr = bytes_to_hex(message, pre="")

In [7]:
from cryptography.hazmat.primitives import padding

block_size_bits = 128

padder = padding.PKCS7(block_size_bits).padder()
message_padded = padder.update(message) + padder.finalize()

print(f"message '{message}' is {len(message)} bytes or {len(bin_repr)} bits")
print(f"block is of {int(block_size_bits/8)} bytes or {block_size_bits} bits\n")
print(f"padded_data: \n\t{message_padded}\nlen_padded_data: \n\t{len(message_padded)}")
#print(f"padded_data: {hexadecimal(int(message))}")

message 'b'Cryptography is a complex subject after all...'' is 46 bytes or 368 bits
block is of 16 bytes or 128 bits

padded_data: 
	b'Cryptography is a complex subject after all...\x02\x02'
len_padded_data: 
	48


Recall that the bytes added to the message is the number of bytes we were missing to get 16 bytes:

In [8]:
print(f"This is, {message_padded[15]} bytes")

This is, 32 bytes


## Encrypting using Advanced Encryption Standard algorithm (AES)

In [9]:
secret_key = os.urandom(32)

cipher = Cipher(algorithms.AES(secret_key), modes.ECB(), backend=default_backend())

encryptor = cipher.encryptor()
decryptor = cipher.decryptor()

In [10]:
ctx = encryptor.update(message_padded) + encryptor.finalize()
print(f"Decrypted message:\n{decryptor.update(ctx) + decryptor.finalize()}")

Decrypted message:
b'Cryptography is a complex subject after all...\x02\x02'


## The mode of operation

A block cipher by itself is only suitable for the secure cryptographic transformation (encryption or decryption) of one fixed-length group of bits called a block. A mode of operation describes how to repeatedly apply a cipher's single-block operation to securely transform amounts of data larger than a block ([Wikipedia](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation)).

The first mode is "not doing anything", this is the Electronic Codebook node, in ```cryptography``` package is implemented in ```cryptography.hazmat.primitives.ciphers.ECB``` function (we've seen in the previous example!).

In [11]:
secret_key = os.urandom(32)

cipher = Cipher(algorithms.AES(secret_key), modes.ECB(), backend=default_backend())

encryptor = cipher.encryptor()
decryptor = cipher.decryptor()

In [12]:
ctx = encryptor.update(message_padded+message_padded) + encryptor.finalize()
print(ctx)

b'Z\xdd&a\xbeTj\xd7\xa9pu!\xdcp\xeb\x84\xb6\xd1k\xa1\x83\xaf\xae\xe3\xd6\t\xce\xbc\xc6\xc3 {VM\xe3\xcf\xa1e\xa2\xe3\x04\x03\xcb\x19&\xac\xc6VZ\xdd&a\xbeTj\xd7\xa9pu!\xdcp\xeb\x84\xb6\xd1k\xa1\x83\xaf\xae\xe3\xd6\t\xce\xbc\xc6\xc3 {VM\xe3\xcf\xa1e\xa2\xe3\x04\x03\xcb\x19&\xac\xc6V'


In [13]:
secret_key = os.urandom(32)

# initialization vector
# https://en.wikipedia.org/wiki/Initialization_vector
iv = os.urandom(16)

cipher = Cipher(algorithms.AES(secret_key), modes.CBC(iv), backend=default_backend())

encryptor = cipher.encryptor()
decryptor = cipher.decryptor()

In [14]:
ctx = encryptor.update(message_padded+message_padded) + encryptor.finalize()
print(ctx)

b'\x01#\xcc\xbc\x07\x1f-;\x01\x19\xd6\xa9\xbdk\xc5\x02\x07\xc5\xd4\xd8\xdc:\x9e\xba\xbf\x04\xffH\xf9\xe5\x1fQ\xc3\x92#\x87\xde\xe1\xefVi\xa5\xdc\xb1\x91m\xa9\xb6\xbe\x83\xb6\xc0/\x03\x83\x01\r\x8b\x94\xd2hz\xa9!c)\xf0\xd2\x15\xe3o8\xbb\xc1\x06|C*\xa25e\xdf\x8b\xd1\xabP\xcax\xa2\xd2\xe2n\xb7,x\xa9'


In [15]:
block_size_bits = 128

for message_len in range(128):
    m = str.encode("a"*message_len)
    
    padder = padding.PKCS7(block_size_bits).padder()
    m_padded = padder.update(m) + padder.finalize()
    encryptor = cipher.encryptor()
    
    ctx = encryptor.update(m_padded) + encryptor.finalize()
    print(len(m_padded))

16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
48
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
80
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
96
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128


## Fernet

In [16]:
from cryptography.fernet import Fernet

# secret key generation
secret_key = Fernet.generate_key()
box = Fernet(secret_key)

max_len = 100
for n in range(1, max_len):
    # generate messages of n a's
    message = "".join(["a" for _ in range(n)])
    message = str.encode(message)
    
    ciphertext = box.encrypt(message)
    print(f"len_message: {len(message)}, len_ciphertext: {len(ciphertext)}")


len_message: 1, len_ciphertext: 100
len_message: 2, len_ciphertext: 100
len_message: 3, len_ciphertext: 100
len_message: 4, len_ciphertext: 100
len_message: 5, len_ciphertext: 100
len_message: 6, len_ciphertext: 100
len_message: 7, len_ciphertext: 100
len_message: 8, len_ciphertext: 100
len_message: 9, len_ciphertext: 100
len_message: 10, len_ciphertext: 100
len_message: 11, len_ciphertext: 100
len_message: 12, len_ciphertext: 100
len_message: 13, len_ciphertext: 100
len_message: 14, len_ciphertext: 100
len_message: 15, len_ciphertext: 100
len_message: 16, len_ciphertext: 120
len_message: 17, len_ciphertext: 120
len_message: 18, len_ciphertext: 120
len_message: 19, len_ciphertext: 120
len_message: 20, len_ciphertext: 120
len_message: 21, len_ciphertext: 120
len_message: 22, len_ciphertext: 120
len_message: 23, len_ciphertext: 120
len_message: 24, len_ciphertext: 120
len_message: 25, len_ciphertext: 120
len_message: 26, len_ciphertext: 120
len_message: 27, len_ciphertext: 120
len_messag